<a id='top'></a>

# 第2章 小売店のデータでデータ加工を行う 10本ノック

In [1]:
# ライブラリのインポート
import numpy as np
import pandas as pd
from IPython.display import display

## 目次
---
[ノック11: データを読み込んでみよう](#11)  
[ノック12; データの揺れを見てみよう](#12)  
[ノック13: データに揺れがあるまま集計してみよう](#13)  
[ノック14: 商品名の揺れを補正しよう](#14)  
[ノック15: 金額欠損値の補完をしよう](#15)  
[ノック16: 顧客名の揺れを補正しよう](#16)  
[ノック17: 日付の揺れを補正しよう](#17)  
[ノック18: 顧客名をキーに2つのデータを結合(ジョイン)しよう](#18)  
[ノック19: クレンジングしたデータをダンプしよう](#19)  
[ノック20: データを集計しよう](#20)

<a id='11'></a>

## ノック11: データを読み込んでみよう

In [2]:
directory = "../data/サンプルコード_20201021/2章/"
uriage = pd.read_csv(directory + "uriage.csv", parse_dates=[0])
display(uriage.head())

kokyaku_daicho = pd.read_excel(directory + "kokyaku_daicho.xlsx", engine="openpyxl")
display(kokyaku_daicho.head())

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二


,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127


<a id='12'></a>

## ノック12; データの揺れを見てみよう

In [3]:
display(uriage["item_name"].head().to_frame())
display(uriage["item_price"].head().to_frame())

,item_name
0,商品A
1,商 品 S
2,商 品 a
3,商品Z
4,商品a


,item_price
0,100.0
1,NaN
2,NaN
3,2600.0
4,NaN


<a id='13'></a>

## ノック13: データに揺れがあるまま集計してみよう

In [4]:
uriage["purchase_month"] = uriage["purchase_date"].dt.strftime("%Y-%m")
pd.pivot_table(uriage, "item_price", "purchase_month", "item_name", ["size", "sum"], 0)

size                                                   ...  \
item_name        商品W  商 品 n  商品E  商品M  商品P  商品S  商品W  商品X 商  品O 商  品Q  ...   
purchase_month                                                         ...   
2019-01            0      1    0    0    0    0    0    0     0     0  ...   
2019-02            0      0    0    0    0    0    0    1     0     0  ...   
2019-03            0      0    1    1    1    0    0    0     0     0  ...   
2019-04            1      0    0    0    0    0    0    0     0     1  ...   
2019-05            0      0    0    0    0    1    0    0     0     0  ...   
2019-06            0      0    0    0    0    0    1    0     0     0  ...   
2019-07            0      0    0    0    0    0    0    0     1     0  ...   

                 sum                                                        
item_name        商品k   商品l   商品o   商品p   商品r   商品s   商品t   商品v   商品x   商品y  
purchase_month                                                              
2019-01         1100  1200  1500     0     0     0     0     0     0     0  
2019-02            0     0     0     0     0  1900  2000  2200     0     0  
2019-03            0     0     0     0     0     0     0     0     0     0  
2019-04            0     0     0     0     0  1900     0     0     0     0  
2019-05            0  1200     0     0     0     0     0     0     0  2500  
2019-06            0     0     0  1600     0     0     0     0  2400     0  
2019-07            0     0  1500     0  1800     0     0     0     0     0  

[7 rows x 198 columns]

<a id='14'></a>

## ノック14: 商品名の揺れを補正しよう

In [5]:
# 商品名のユニーク数
uriage["item_name"].nunique()

99

In [6]:
# 表記揺れ補正
uriage["item_name"] = uriage["item_name"].str.replace(" ", "").str.upper()
uriage["item_name"]

0       商品A
1       商品S
2       商品A
3       商品Z
4       商品A
       ... 
2994    商品Y
2995    商品M
2996    商品Q
2997    商品H
2998    商品D
Name: item_name, Length: 2999, dtype: object

In [7]:
# 確認
print(uriage["item_name"].nunique())
np.sort(uriage["item_name"].unique())

26


array(['商品A', '商品B', '商品C', '商品D', '商品E', '商品F', '商品G', '商品H', '商品I',
       '商品J', '商品K', '商品L', '商品M', '商品N', '商品O', '商品P', '商品Q', '商品R',
       '商品S', '商品T', '商品U', '商品V', '商品W', '商品X', '商品Y', '商品Z'],
      dtype=object)

<a id='15'></a>

## ノック15: 金額欠損値の補完をしよう

In [8]:
# 欠損値の確認
uriage.isna().any()

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

In [9]:
# 欠損値補完
for item in uriage["item_name"].unique():
    
    # 商品ごとに単価を調べて補完
    price = uriage.query("item_name == '{}'".format(item))["item_price"].max()
    uriage.loc[(uriage["item_price"].isna()) & (uriage["item_name"] == item), "item_price"] = price

# 整数値に変換
uriage["item_price"] = uriage["item_price"].astype(int)

# 確認
print(uriage.isna().any())
display(uriage.head())

purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_month    False
dtype: bool


,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100,深井菜々美,2019-06
1,2019-07-13 13:05:29,商品S,1900,浅田賢二,2019-07
2,2019-05-11 19:42:07,商品A,100,南部慶二,2019-05
3,2019-02-12 23:40:45,商品Z,2600,麻生莉緒,2019-02
4,2019-04-22 03:09:35,商品A,100,平田鉄二,2019-04


<a id='16'></a>

## ノック16: 顧客名の揺れを補正しよう

In [10]:
display(kokyaku_daicho["顧客名"].head().to_frame())
display(uriage["customer_name"].head().to_frame())

,顧客名
0,須賀ひとみ
1,岡田 敏也
2,芳賀 希
3,荻野 愛
4,栗田 憲一


,customer_name
0,深井菜々美
1,浅田賢二
2,南部慶二
3,麻生莉緒
4,平田鉄二


In [11]:
# スペース削除
kokyaku_daicho["顧客名"] = kokyaku_daicho["顧客名"].str.replace(" ", "").str.replace("　", "")
kokyaku_daicho["顧客名"].head()

0    須賀ひとみ
1     岡田敏也
2      芳賀希
3      荻野愛
4     栗田憲一
Name: 顧客名, dtype: object

<a id='17'></a>

## ノック17: 日付の揺れを補正しよう

In [12]:
kokyaku_daicho["登録日"]

0      2018/01/04
1           42782
2      2018/01/07
3           42872
4           43127
          ...    
195    2017/06/20
196    2018/06/20
197    2017/04/29
198    2019/04/19
199    2019/04/23
Name: 登録日, Length: 200, dtype: object

In [13]:
# 数値を抽出
is_digit = kokyaku_daicho["登録日"].astype(str).str.isdigit()
is_digit.sum()

22

In [14]:
# 数値を日数として日付に変換
kokyaku_daicho.loc[is_digit, "登録日"] = pd.to_timedelta(
    kokyaku_daicho.loc[is_digit, "登録日"] - 2, unit="D") + pd.to_datetime("1900/1/1")
kokyaku_daicho["登録日"] = pd.to_datetime(kokyaku_daicho["登録日"])
kokyaku_daicho["登録日"]

0     2018-01-04
1     2017-02-16
2     2018-01-07
3     2017-05-17
4     2018-01-27
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 登録日, Length: 200, dtype: datetime64[ns]

In [15]:
# 確認
is_digit = kokyaku_daicho["登録日"].astype(str).str.isdigit()
is_digit.sum()

0

In [16]:
# 登録年月作成
kokyaku_daicho["登録年月"] = kokyaku_daicho["登録日"].dt.strftime("%Y-%m")
kokyaku_daicho

,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04,2018-01
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-16,2017-02
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07,2018-01
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-17,2017-05
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-27,2018-01
...,...,...,...,...,...,...
195,川上りえ,かわかみ りえ,G市,kawakami_rie@example.com,2017-06-20,2017-06
196,小松季衣,こまつ としえ,E市,komatsu_toshie@example.com,2018-06-20,2018-06
197,白鳥りえ,しらとり りえ,F市,shiratori_rie@example.com,2017-04-29,2017-04
198,大西隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019-04-19,2019-04


In [17]:
# 確認
resister = kokyaku_daicho.groupby("登録年月")["顧客名"].count().rename("登録数").to_frame()
display(resister)
print(resister.sum())
print(len(kokyaku_daicho))

,登録数
登録年月,
2017-01,15
2017-02,11
2017-03,14
2017-04,15
2017-05,14
2017-06,13
2017-07,17
2018-01,13
2018-02,15


登録数    200
dtype: int64
200


<a id='18'></a>

## ノック18: 顧客名をキーに2つのデータを結合(ジョイン)しよう

In [18]:
join_data = pd.merge(uriage, kokyaku_daicho, how="left", left_on="customer_name", right_on="顧客名"
                    ).drop(columns="customer_name")
join_data

,purchase_date,item_name,item_price,purchase_month,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,2019-06-13 18:02:34,商品A,100,2019-06,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26,2017-01
1,2019-07-13 13:05:29,商品S,1900,2019-07,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07,2018-04
2,2019-05-11 19:42:07,商品A,100,2019-05,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19,2018-06
3,2019-02-12 23:40:45,商品Z,2600,2019-02,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22,2018-07
4,2019-04-22 03:09:35,商品A,100,2019-04,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07,2017-06
...,...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:39,商品Y,2500,2019-02,福島友也,ふくしま ともや,B市,fukushima_tomoya@example.com,2017-07-01,2017-07
2995,2019-06-22 04:03:43,商品M,1300,2019-06,大倉晃司,おおくら こうじ,E市,ookura_kouji@example.com,2018-03-31,2018-03
2996,2019-03-29 11:14:05,商品Q,1700,2019-03,尾形小雁,おがた こがん,B市,ogata_kogan@example.com,2017-03-15,2017-03
2997,2019-07-14 12:56:49,商品H,800,2019-07,芦田博之,あしだ ひろゆき,E市,ashida_hiroyuki@example.com,2018-07-13,2018-07


<a id='19'></a>

## ノック19: クレンジングしたデータをダンプしよう

In [19]:
# 列の並び替え
join_data = join_data[["purchase_date", "purchase_month", "item_name", "item_price",
                       "顧客名", "かな", "地域", "メールアドレス", "登録日", "登録年月"]]
join_data.head()

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,2019-06-13 18:02:34,2019-06,商品A,100,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26,2017-01
1,2019-07-13 13:05:29,2019-07,商品S,1900,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07,2018-04
2,2019-05-11 19:42:07,2019-05,商品A,100,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19,2018-06
3,2019-02-12 23:40:45,2019-02,商品Z,2600,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22,2018-07
4,2019-04-22 03:09:35,2019-04,商品A,100,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07,2017-06


In [20]:
# to csv
join_data.to_csv(directory + "dump_data.csv", index=False)

<a id='20'></a>

## ノック20: データを集計しよう

In [21]:
# データ読込
join_data = pd.read_csv(directory + "dump_data.csv", parse_dates=[0, 8])
join_data.head()

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,2019-06-13 18:02:34,2019-06,商品A,100,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26,2017-01
1,2019-07-13 13:05:29,2019-07,商品S,1900,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07,2018-04
2,2019-05-11 19:42:07,2019-05,商品A,100,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19,2018-06
3,2019-02-12 23:40:45,2019-02,商品Z,2600,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22,2018-07
4,2019-04-22 03:09:35,2019-04,商品A,100,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07,2017-06


In [22]:
# 月ごとの商品購入数の集計
join_data.pivot_table(index="purchase_month", columns="item_name", aggfunc="size", fill_value=0)

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
2019-01,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
2019-02,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
2019-03,17,21,20,17,9,27,14,18,12,16,...,23,16,20,12,23,18,16,21,16,0
2019-04,17,19,24,20,18,17,14,11,18,13,...,20,20,16,16,11,15,14,16,20,0
2019-05,24,14,16,14,19,18,23,15,16,11,...,13,22,18,16,16,9,21,16,20,0
2019-06,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
2019-07,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


In [23]:
# 月ごとの売上の集計
join_data.pivot_table("item_price", "purchase_month", "item_name", "sum", fill_value=0)

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
2019-01,1800,2600,5700,6800,9000,9000,7700,12800,16200,17000,...,28900,37800,38000,34000,14700,48400,29900,33600,25000,0
2019-02,1900,2800,7800,8400,8000,8400,9800,13600,10800,14000,...,37400,39600,41800,46000,39900,48400,55200,38400,27500,2600
2019-03,1700,4200,6000,6800,4500,16200,9800,14400,10800,16000,...,39100,28800,38000,24000,48300,39600,36800,50400,40000,0
2019-04,1700,3800,7200,8000,9000,10200,9800,8800,16200,13000,...,34000,36000,30400,32000,23100,33000,32200,38400,50000,0
2019-05,2400,2800,4800,5600,9500,10800,16100,12000,14400,11000,...,22100,39600,34200,32000,33600,19800,48300,38400,50000,0
2019-06,2400,2400,3300,7600,6500,10800,10500,10400,17100,22000,...,25500,28800,39900,24000,37800,44000,39100,36000,32500,0
2019-07,2000,4000,5100,6800,6000,10200,13300,15200,17100,23000,...,25500,34200,43700,42000,27300,61600,36800,43200,30000,0


In [24]:
# 月ごとの購入人数の集計
join_data.pivot_table(index="purchase_month", columns="顧客名", aggfunc="size", fill_value=0)

顧客名,さだ千佳子,中仁晶,中田美智子,丸山光臣,久保田倫子,亀井一徳,五十嵐春樹,井上桃子,井口寛治,井川真悠子,...,香椎優一,高原充則,高梨結衣,高沢美咲,高田さんま,鳥居広司,鶴岡薫,麻生莉緒,黄川田博之,黒谷長利
purchase_month,,,,,,,,,,,,,,,,,,,,,
2019-01,3,1,4,2,2,0,5,3,3,1,...,0,1,1,1,5,2,0,2,2,5
2019-02,9,1,2,2,1,4,2,1,0,4,...,4,0,3,2,0,1,2,4,0,1
2019-03,1,2,1,6,1,4,3,3,2,2,...,3,1,6,2,4,2,4,2,2,1
2019-04,0,3,1,2,0,2,2,0,3,2,...,2,4,2,3,4,3,2,1,2,0
2019-05,3,2,5,2,4,1,2,1,3,3,...,1,1,1,0,2,2,3,4,4,1
2019-06,1,3,0,4,1,1,1,2,2,3,...,7,3,0,2,1,0,2,1,2,4
2019-07,3,0,3,2,5,3,5,2,5,5,...,2,4,4,2,0,2,4,3,4,1


In [25]:
# 月、地域ごとの購入数の集計
join_data.pivot_table(index="purchase_month", columns="地域", aggfunc="size", fill_value=0)

地域,A市,B市,C市,D市,E市,F市,G市,H市
purchase_month,,,,,,,,
2019-01,59,55,72,34,49,57,49,42
2019-02,71,46,65,48,61,52,43,63
2019-03,64,52,57,43,52,59,51,59
2019-04,64,48,54,45,48,58,40,52
2019-05,57,52,68,48,59,65,35,43
2019-06,53,47,61,30,51,51,58,58
2019-07,76,53,61,42,54,64,47,54


In [27]:
# 期間中に購入履歴の無い顧客
isna = pd.merge(uriage, kokyaku_daicho, how="outer", left_on="customer_name", right_on="顧客名"
               ).drop(columns="customer_name")
isna = isna.loc[isna.isna().any(axis=1)].dropna(axis=1)
isna

,顧客名,かな,地域,メールアドレス,登録日,登録年月
2999,福井美希,ふくい みき,D市,fukui_miki1@example.com,2019-04-23,2019-04


[<div style="text-align: right;">トップに戻る</div>](#top)